In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet,SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,HistGradientBoostingRegressor
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import cross_validate
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import altair as alt

def save(model, filename='bestmodel.pickle'):
    with open('output/'+filename, 'wb') as handle:
        pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

def submit(model):
    pred = model.predict(final_test)
    final_test['SalePrice'] = np.exp(pred)
    final_test[['Id','SalePrice']].to_csv('output/submission.csv', index=False)

## Import data

In [2]:
f = open("output/engineered_datasets.pickle","rb")
train_x, train_y, final_test, num_x, cat_x, cat_x_ind = pickle.load(f)

# cat_x_ind = [ind for ind,name in enumerate(train_x.columns) if name in cat_x ]
# cat_x_ind = list(range(37,80))



In [3]:
from sklearn.preprocessing import LabelEncoder
ord_x = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

In [4]:
def inv_y(transformed_y):
    return np.exp(transformed_y)


## Make preprocessing pipelines

In [5]:
from sklearn.impute import SimpleImputer
# todo multivariate imputation, possibly with pipelines for numeric and categorical data

from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
# https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
# https://stackoverflow.com/questions/51237635/difference-between-standard-scaler-and-minmaxscaler/51237727
# don't know features are normal so just going with minmax scalar atm

from sklearn.preprocessing import OneHotEncoder
#https://stackoverflow.com/questions/36631163/what-are-the-pros-and-cons-between-get-dummies-pandas-and-onehotencoder-sciki
#The crux of it is that the sklearn encoder creates a function which persists and can then be applied to new data sets which use the same categorical variables, with consistent results.
# So don't use pandas get dummies, but a OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

numeric_transformer = Pipeline(steps=[
    ('impute_num', SimpleImputer(strategy='median')),
    ('scale_num', StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
    ('impute_cat', SimpleImputer(strategy='most_frequent')),
    # ('label_encode', preprocessing.LabelEncoder()),
    ('onehot_cat', OneHotEncoder(handle_unknown="ignore"))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, num_x),
        ('category', categorical_transformer, cat_x)])


def densify(x): # needs to use a function, lambda gives problems with pickling
    return x.todense()


In [6]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.base import TransformerMixin

class OutlierExtractor(TransformerMixin):
    def __init__(self, **kwargs):
        """
        Create a transformer to remove outliers. A threshold is set for selection
        criteria, and further arguments are passed to the LocalOutlierFactor class

        Keyword Args:
            neg_conf_val (float): The threshold for excluding samples with a lower
               negative outlier factor.

        Returns:
            object: to be used as a transformer method as part of Pipeline()
        """

        self.threshold = kwargs.pop('neg_conf_val', -10.0)

        self.kwargs = kwargs

    def transform(self, X, y):
        """
        Uses LocalOutlierFactor class to subselect data based on some threshold

        Returns:
            ndarray: subsampled data

        Notes:
            X should be of shape (n_samples, n_features)
        """
        X = np.asarray(X)
        y = np.asarray(y)
        lcf = LocalOutlierFactor(**self.kwargs)
        lcf.fit(X)
        return (X[lcf.negative_outlier_factor_ > self.threshold, :],
                y[lcf.negative_outlier_factor_ > self.threshold])

    def fit(self, *args, **kwargs):
        return self

In [19]:

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


def get_pipeline(model, outlier = False, impute=True,scale=True,onehot=True,label=False,to_dense=False):
    if outlier:
        print('outlier removal not implemented yet')
        return
    cat_steps = []
    if impute: cat_steps.append(('impute_cat', SimpleImputer(strategy='most_frequent')))
    # if impute: cat_steps.append(('impute_cat', IterativeImputer(max_iter=10, random_state=0)))
    if label: cat_steps.append(('label_encode', preprocessing.LabelEncoder()))
    if onehot: cat_steps.append(('onehot_cat', OneHotEncoder(handle_unknown="ignore")))
    categorical_transformer = Pipeline(steps=cat_steps)

    num_steps = []
    if impute: num_steps.append(('impute_num', SimpleImputer(strategy='mean')))
    # if impute: num_steps.append(('impute_num', IterativeImputer(max_iter=10, random_state=0)))
    if scale: num_steps.append(('scale_num', MinMaxScaler()))
    numeric_transformer = Pipeline(steps=num_steps)

    preprocessor = ColumnTransformer(transformers=[
        ('numeric', numeric_transformer, num_x),
        ('category', categorical_transformer, cat_x),
        # ('labeling',LabelEncoder(),ord_x)
        ])

    final_pipe = [('preprocess', preprocessor)]
    if to_dense: final_pipe.append(('to_dense',FunctionTransformer(densify, accept_sparse=True)))
    final_pipe.append(('model',model))

    return Pipeline(steps=final_pipe)
    

## Collection of models with hyperparameters

In [9]:
from skopt.space import Real, Categorical, Integer
from dataclasses import dataclass,field
@dataclass
class Param:
     model: object = None
     hyper: dict = None
     preprocess: dict = field(default_factory=lambda: {})
     
models = {    
            'LinearRegression':Param(LinearRegression(),
                {}),

            'Lasso':Param(Lasso(),
                {'alpha':(0.00001,1.0,'log-uniform')}),

            'Ridge': Param(Ridge(),
                {'alpha':(0.00001,1.0,'log-uniform')}),

            'ElasticNet':Param(ElasticNet(),
                {'l1_ratio': Real(0.01, 1.0, 'log-uniform'),
                'alpha':Real(0.0001, 1.0, 'log-uniform')
                }),

            'svm.SVR': Param(svm.SVR(), 
                # https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html#advanced-example
                # https://scikit-learn.org/stable/modules/svm.html#regression
                {'C': Real(0.1, 20, 'log-uniform'),
                'degree': Integer(1, 8),  # integer valued parameter
                'epsilon': Real(0.004, 0.01),  # integer valued parameter
                'gamma': Real(0, 0.001),  # integer valued parameter
                'kernel': ['linear', 'rbf']}),

            'KNeighborsRegressor':Param(KNeighborsRegressor(),
                {'n_neighbors': (2,3,4,5,6), 
                'weights': ['uniform','distance']}),

            'RandomForestRegressor':Param(RandomForestRegressor(),
                {'n_estimators' : (1,100, 'log-uniform'), # gamble
                'max_depth' : (3, 40,'log-uniform')}),

            # SGD does not work atm with bayessearch (not hashable)
            # 'SGDRegressor':Param(SGDRegressor(),
            #     {'loss' : ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
            #     'penalty' : ['l1', 'l2', 'elasticnet'],
            #     'alpha' : (0.0,1000,'log-uniform'),
            #     'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive'],
            #     'class_weight' : [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}],
            #     'eta0' : [1, 10, 100]}),

            # Catboost does not work atm with bayessearch. Tried to define __hash__, but then got into problems with is_comparable
            'CatBoostRegressor':Param(CatBoostRegressor(silent=True,one_hot_max_size=20, iterations = 300, cat_features = cat_x_ind),
                {
                # 'iterations': Integer(350,350),
                 'depth': Integer(4, 10),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),
                #  'scale_pos_weight':Real(0.01, 1.0, 'uniform')
                 },
            preprocess={'onehot':False}),

            'xgb.XGBRegressor':Param(xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1, silent=True),
                {'max_depth': Integer(2, 10,'log-uniform'), #9,12
                'min_child_weight': Integer(0, 4,'uniform'), # if leaves with small amount of observations are allowed?
                'gamma' : Real(0,0.1), # these 3 for model complexity. gemma is a threshold for gain of the new split. 
                # 'subsample': (1,),
                'colsample_bytree' : Real(0.2,1.0,'log-uniform'), # these 3 for making model more robust to noise
                'reg_lambda' : Real(0.0,0.9,'uniform'), # regularization lambda, reduces similarity scores and therefore lowers gain. reduces sensitivity to individual observations
                'colsample_bylevel': Real(0.7,1.0,'log-uniform'),
                'learning_rate': Real(0.001,0.4,'log-uniform'), # 0.3 is default
                'max_delta_step': Real(0.0,10.0,'uniform'),
                'n_estimators': Integer(10,2500,'log-uniform')}), # number of trees to build
            #https://www.kaggle.com/c/LANL-Earthquake-Prediction/discussion/89994

            'lgb.LGBMRegressor': Param(lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=500,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11,categorical_feature= cat_x_ind),
                        # num_leaves=31,
                        # learning_rate=0.05,
                        # n_estimators=20,
                {'num_leaves': Integer(3, 8),
                'n_estimators':Integer(100, 1000),
                'min_data_in_leaf': Integer(3, 30),
                'max_depth': Integer(3, 12),
                'learning_rate': Real(0.01, 0.3,'log_uniform'), #'log_uniform'
                'bagging_freq': Integer(3, 7),
                'bagging_fraction': Real(0.6, 0.95, 'log_uniform'),
                'reg_alpha': Real(0.1, 0.95, 'log_uniform'),
                'reg_lambda': Real(0.1, 0.95, 'log_uniform')
            }),

            # A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array
            'HistGradientBoostingRegressor':Param(HistGradientBoostingRegressor(loss='huber'),
                {'loss':'huber'}),
            'GradientBoostingRegressor': Param(GradientBoostingRegressor(learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5),
                {
                'max_depth': Integer(3,12),
                'min_samples_leaf': Integer(5,20),
                }),

            'LassoCV':Param(LassoCV(),
                {}),
                
            'MLPRegressor':Param(MLPRegressor(),
                {})
                }

## Rough test of all models

In [10]:
def cross_val_models(to_test):
    for name in to_test:
        pipe = get_pipeline(models[name].model, **models[name].preprocess)
        # score = -1 * cross_val_score(pipe, train_x, train_y,cv=3,scoring='neg_root_mean_squared_error')
        # below is necessary for looking at train scores
        num_fold = 5
        scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
        # scoring is identical to 
        # make_scorer(mean_squared_error,greater_is_better=False, root=False,squared=False)
        # neg_mean_squared_log_error
        print(f"{name:<30}train {-1 * sum(scores['train_score'])/num_fold:.5f}, test {-1 * sum(scores['test_score'])/num_fold:.5f}")

# def test_model(model):
#         num_fold = 5
#         pipe = get_pipeline(model)
#         scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
#         print(f"train {-1 * sum(scores['train_score'])/num_fold:.3f}, test {-1 * sum(scores['test_score'])/num_fold:.3f}")
#         return pipe

def test_model(model,param=None):
        num_fold = 5
        if not param: param = {}
        pipe = get_pipeline(model,**param)
        scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
        print(f"test {-1 * sum(scores['test_score'])/num_fold:.5f}, train {-1 * sum(scores['train_score'])/num_fold:.5f}")
        return pipe

In [112]:
cross_val_models(models)

LinearRegression              train 0.08575, test 0.12940
Lasso                         train 0.38104, test 0.38317
Ridge                         train 0.08779, test 0.11919
ElasticNet                    train 0.37863, test 0.38132
svm.SVR                       train 0.20313, test 0.22196
KNeighborsRegressor           train 0.17845, test 0.22384
RandomForestRegressor         train 0.05090, test 0.13258
CatBoostRegressor             train 0.04411, test 0.11473
xgb.XGBRegressor              train 0.07702, test 0.11371
lgb.LGBMRegressor             train 0.07283, test 0.11488
HistGradientBoostingRegressor train nan, test nan
GradientBoostingRegressor     train 0.09853, test 0.12320
LassoCV                       train 0.11813, test 0.12650
MLPRegressor                  train 0.25308, test 1.53013


## Tune number of iterations for Catboost

In [29]:
model = models['CatBoostRegressor'].model
pipe = test_model(model, models['CatBoostRegressor'].preprocess)
fitted = pipe.named_steps['model'].fit(pipe[:-1].transform(train_x[:1000]),train_y[:1000],eval_set=(
    pipe[:-1].transform(train_x[1000:]),train_y[1000:]))

In [30]:
fits = {}
fits['learn'] = list(fitted.get_evals_result()['learn'].values())[0]
fits['validation'] = list(fitted.get_evals_result()['validation'].values())[0]
df = pd.DataFrame.from_dict(fits)
df.reset_index(inplace=True)
df =  df.melt('index',var_name= 'type',value_name='score')
fitted.get_evals_result()

alt.Chart(df).mark_line().encode(
    x='index',
    y='score',
    color='type',
)

alt.Chart(...)

## Hyperparameter tuning

In [14]:
from skopt import BayesSearchCV, callbacks
from skopt.space import Real, Categorical, Integer

TRAIN_TIME = 600
NUM_ITERATIONS = 150
NO_IMPROVEMENT_STOP_THRES = 25

def gen_opt_settings(model_name):
    model = {'model': [models[model_name].model]}
    for k,v in models[model_name].hyper.items():
        model['model__'+k] = v
    return (model, NUM_ITERATIONS)

def optimize_model(model_name):
    print('running', model_name)
    def no_improvement_detector(optim_result):
        score = opt.best_score_
        # print(optim_result.x)
        print(f"{'best score':15}{score}")
        if score > opt.train_status['current_score']:
            opt.train_status['current_score'] = score
            opt.train_status['not_improving'] = 0
        else:
            opt.train_status['not_improving'] += 1
            if opt.train_status['not_improving'] == opt.train_status['stop_thres']: return True
    checkpointsaver = callbacks.CheckpointSaver("output/" + model_name + "_skopt.pkl")
    deadlinestopper = callbacks.DeadlineStopper(TRAIN_TIME)

    opt = BayesSearchCV(
        get_pipeline(models[model_name].model, **models[model_name].preprocess),
        [gen_opt_settings(model_name)],
        cv=5, 
        scoring = 'neg_root_mean_squared_error',
        return_train_score = True,
        random_state = 112 
        )
    opt.train_status = { 'current_score': -100, 'not_improving': 0, 'stop_thres' :NO_IMPROVEMENT_STOP_THRES}

    opt.fit(train_x,train_y, callback = [no_improvement_detector,checkpointsaver,deadlinestopper])
    return opt

def hashing(self): return 8398398478478 
CatBoostRegressor.__hash__ = hashing

to_test = [k for k in models]
to_test = [
    # 'svm.SVR',
    'ElasticNet',
    'RandomForestRegressor',
    'CatBoostRegressor',
    'xgb.XGBRegressor',
    'lgb.LGBMRegressor',
    'GradientBoostingRegressor']

results = {}
for name in to_test:
    results[name] = optimize_model(name)

running ElasticNet
best score     -0.25420393613597186
best score     -0.11985911658019094
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923
best score     -0.1075787060493923

KeyboardInterrupt: 

In [16]:
#summarize tuning results
for model in results:
    best_run = results[model].cv_results_['rank_test_score'].index(1)
    mean_test_score = -1 * results[model].cv_results_['mean_test_score'][best_run]
    std_test_score = results[model].cv_results_['std_test_score'][best_run]
    mean_train_score = -1 * results[model].cv_results_['mean_train_score'][best_run]
    mean_score_time = results[model].cv_results_['mean_score_time'][best_run]
    best_params = results[model].best_params_

    # print(f"{model:<30} Best score {mean_test_score:.4f} std {std_test_score:.4f} train {mean_train_score:.4f} time {mean_score_time:.4f}")
    print(f"{model:<30} Best score {mean_test_score:.4f} std {std_test_score:.4f} train {mean_train_score:.4f} time {mean_score_time:.4f}")

ElasticNet                     Best score 0.1076 std 0.0057 train 0.0946 time 0.0552
RandomForestRegressor          Best score 0.1314 std 0.0047 train 0.0522 time 0.0634
CatBoostRegressor              Best score 0.1174 std 0.0036 train 0.0809 time 0.0574
xgb.XGBRegressor               Best score 0.1127 std 0.0050 train 0.0757 time 0.0324
lgb.LGBMRegressor              Best score 0.1142 std 0.0053 train 0.0726 time 0.0264


In [13]:
# Assert to see if it matches the BayesSearchCV results. Not necessary since it matches
for model in results:
    pipe = results[model].best_estimator_
    num_fold = 5
    scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
    print(f"{model:<30}train {-1 * sum(scores['train_score'])/num_fold:.4f}, test {-1 * sum(scores['test_score'])/num_fold:.4f}")


svm.SVR                       train 0.0961, test 0.1163
ElasticNet                    train 0.0956, test 0.1080
RandomForestRegressor         train 0.0504, test 0.1306
CatBoostRegressor             train 0.0810, test 0.1189
xgb.XGBRegressor              train 0.0796, test 0.1132
lgb.LGBMRegressor             train 0.0734, test 0.1139


## Experimental runs

In [18]:
f = open("output/engineered_datasets.pickle","rb")
train_x, train_y, final_test, num_x, cat_x, cat_x_ind = pickle.load(f)
model = test_model(Lasso(alpha=0.00035045133983294347, copy_X=True, fit_intercept=True,
                    max_iter=1000, normalize=False, positive=False, precompute=False,
                    random_state=42, selection='cyclic', tol=0.0001, warm_start=False))

test 0.12664, train 0.09477


In [43]:
model = test_model(LassoCV())

train 0.09580, test 0.10812


## Stacking best models from hyperparam

In [29]:
from sklearn.ensemble import StackingRegressor

to_stack_list = [
    'ElasticNet',
    'RandomForestRegressor',
    'CatBoostRegressor',
    'xgb.XGBRegressor',
    'lgb.LGBMRegressor']

to_stack = [(model_name, results[model_name].best_estimator_) for model_name in to_stack_list]

model = StackingRegressor(to_stack, final_estimator = LinearRegression(), passthrough = False)
num_fold = 5
scores = cross_validate(model, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
print(f"stacking model train {-1 * sum(scores['train_score'])/num_fold:.4f}, test {-1 * sum(scores['test_score'])/num_fold:.4f}")

stacking model train nan, test nan


In [21]:
num_fold = 5

print(f"stacking model train {-1 * sum(scores['train_score'])/num_fold:.4f}, test {-1 * sum(scores['test_score'])/num_fold:.4f}")


stacking model train 0.0831, test 0.1062


## Saving

In [24]:
train_info = (train_x, train_y, num_x, cat_x)
with open('output/train_info.pickle', 'wb') as handle:
    pickle.dump(train_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
# model = get_pipeline(CatBoostRegressor(silent=True))
model = model.fit(train_x,train_y)
submit(model)
save(model,'lasso.pickle')